In [4]:
import numpy as np
import pandas as pd
from pdml_data import panel_sim_CP
from doubleml.double_ml_data import DoubleMLClusterData
from sklearn.base import clone
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from doubleml.plm.plpr import DoubleMLPLPR

data = panel_sim_CP(N=250, T=10, p=30, x_var=5, seed=1, a=0.25, b=0.5, dgp='DGP3', theta=0.5)
data.head(11)


,id,time,d,y,x1,x2,x3,x4,x5,x6,...,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30
0,1,1,5.688565,5.575328,-3.844683,0.127726,-1.787843,-0.652025,-0.579103,0.423272,...,3.226371,-1.206765,0.287060,3.936408,2.161247,1.594426,2.920766,-1.351933,1.423444,3.151186
1,1,2,1.803330,0.888238,3.624470,-1.802684,-0.562761,0.855777,-0.646218,-0.876128,...,-0.397198,-3.377483,2.261128,-3.301694,-0.320195,2.309822,-0.497333,3.287378,-1.945398,0.825535
2,1,3,6.503023,7.930423,1.907997,-0.312405,3.099893,1.225655,-3.661449,8.851705,...,1.744194,-1.407474,-2.489719,-0.150733,2.597059,-0.061558,3.905148,-1.733110,0.316718,-5.626626
3,1,4,2.127946,1.944358,-1.331954,-0.691216,1.142492,3.825156,0.078136,3.251059,...,0.754049,2.254103,1.755821,-1.486690,-4.349257,-2.046951,2.739532,-2.355801,1.824729,-1.369384
4,1,5,3.623328,4.710092,0.879019,-4.078409,2.609659,-0.088702,1.980767,0.424544,...,2.202419,2.395393,-2.453312,1.875244,-2.323682,1.639090,-4.245873,-2.497935,-1.138097,-0.372274
5,1,6,8.224220,11.243063,3.183299,2.021404,3.522853,2.698057,-0.632502,-0.595525,...,-4.237946,0.200640,0.917382,1.922730,-2.009517,0.714775,0.711414,-0.043003,0.335447,1.036494
6,1,7,0.537203,-0.312131,0.889688,-2.227148,-2.674028,5.603542,4.292786,-3.111922,...,-1.768282,2.736326,-0.132751,3.240039,-1.067903,0.058138,-3.015777,2.912558,-0.810825,-3.320911
7,1,8,1.054859,1.672320,-1.324784,-5.153931,-0.071145,0.251530,0.644162,3.349872,...,2.160536,1.623199,7.426158,-1.342145,-0.848627,-2.269170,0.974895,-1.537269,-6.033720,-2.713107
8,1,9,1.639310,1.337860,0.161560,2.257953,-3.481427,-1.369462,-0.311600,-1.629059,...,3.441772,-2.568581,-2.438071,3.801736,1.361282,-4.206963,1.111928,0.530692,-4.795123,-0.826367
9,1,10,0.975663,1.928549,-0.039030,1.635466,2.134317,0.214096,2.310866,-0.327077,...,-4.665311,0.809013,0.952386,0.109747,2.464676,-2.749412,2.478942,-1.571778,1.622380,-0.724943


In [3]:
# FD
df = data.copy()

shifted = df.loc[:,~df.columns.isin(['d', 'y', 'time'])].groupby(["id"]).shift(1)

first_diff = df.loc[:,df.columns.isin(['id', 'd', 'y'])].groupby(["id"]).diff()
df_fd = df.join(shifted.rename(columns=lambda x: x +"_lag"))
df_fd = df_fd.join(first_diff.rename(columns=lambda x: x +"_diff"))

df = df_fd.dropna(subset=['x1_lag']).reset_index(drop=True)
df.columns

Index(['id', 'time', 'd', 'y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',
       'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18',
       'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28',
       'x29', 'x30', 'x1_lag', 'x2_lag', 'x3_lag', 'x4_lag', 'x5_lag',
       'x6_lag', 'x7_lag', 'x8_lag', 'x9_lag', 'x10_lag', 'x11_lag', 'x12_lag',
       'x13_lag', 'x14_lag', 'x15_lag', 'x16_lag', 'x17_lag', 'x18_lag',
       'x19_lag', 'x20_lag', 'x21_lag', 'x22_lag', 'x23_lag', 'x24_lag',
       'x25_lag', 'x26_lag', 'x27_lag', 'x28_lag', 'x29_lag', 'x30_lag',
       'd_diff', 'y_diff'],
      dtype='object')

In [4]:
np.random.seed(1)
obj_dml_data_pdml = DoubleMLClusterData(df,
                                        y_col='y_diff',
                                        d_cols='d_diff',
                                        cluster_cols='id',
                                        x_cols=[col for col in df.columns if col.startswith("x")],
                                        use_other_treat_as_covariate=False)

learner = LassoCV()
ml_l = clone(learner)
ml_m = clone(learner)

obj_dml_plr_sim = DoubleMLPLPR(obj_dml_data_pdml, ml_l, ml_m, pdml_approach='transform')
obj_dml_plr_sim.fit()

print(obj_dml_plr_sim)

================== DoubleMLPLPR Object ==================

------------------ Data summary      ------------------
Outcome variable: y_diff
Treatment variable(s): ['d_diff']
Cluster variable(s): ['id']
Covariates: ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30', 'x1_lag', 'x2_lag', 'x3_lag', 'x4_lag', 'x5_lag', 'x6_lag', 'x7_lag', 'x8_lag', 'x9_lag', 'x10_lag', 'x11_lag', 'x12_lag', 'x13_lag', 'x14_lag', 'x15_lag', 'x16_lag', 'x17_lag', 'x18_lag', 'x19_lag', 'x20_lag', 'x21_lag', 'x22_lag', 'x23_lag', 'x24_lag', 'x25_lag', 'x26_lag', 'x27_lag', 'x28_lag', 'x29_lag', 'x30_lag']
Instrument variable(s): None
No. Observations: 2250

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: LassoCV()
Learner ml_m: LassoCV()
Out-of-sample Per

In [5]:
# WD
df = data.copy()

df_demean = df.loc[:,~df.columns.isin(['time'])].groupby(["id"]).transform(lambda x: x - x.mean())

# add xbar (the grand mean allows a consistent estimate of the constant term)
within_means = df_demean + df.loc[:,~df.columns.isin(['id','time'])].mean()

df_wd = df.loc[:,df.columns.isin(['id','time'])]
df = df_wd.join(within_means)
df.columns

Index(['id', 'time', 'd', 'y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',
       'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18',
       'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28',
       'x29', 'x30'],
      dtype='object')

In [6]:
np.random.seed(1)
obj_dml_data_pdml = DoubleMLClusterData(df,
                                        y_col='y',
                                        d_cols='d',
                                        cluster_cols='id',
                                        x_cols=[col for col in df.columns if col.startswith("x")],
                                        use_other_treat_as_covariate=False)

learner = LassoCV()
ml_l = clone(learner)
ml_m = clone(learner)

obj_dml_plr_sim = DoubleMLPLPR(obj_dml_data_pdml, ml_l, ml_m, pdml_approach='transform')
obj_dml_plr_sim.fit()

print(obj_dml_plr_sim)

================== DoubleMLPLPR Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Cluster variable(s): ['id']
Covariates: ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x30']
Instrument variable(s): None
No. Observations: 2500

------------------ Score & algorithm ------------------
Score function: partialling out

------------------ Machine learner   ------------------
Learner ml_l: LassoCV()
Learner ml_m: LassoCV()
Out-of-sample Performance:
Regression:
Learner ml_l RMSE: [[3.75956729]]
Learner ml_m RMSE: [[2.57900455]]

------------------ Resampling        ------------------
No. folds per cluster: 5
No. folds: 5
No. repeated sample splits: 1

------------------ Fit summary       ------------------
       coef   std err           t  P>|t|     2.5 %    97.5 

In [7]:
# CRE
df = data.copy()

id_means = df.loc[:,~df.columns.isin(['time', 'y'])].groupby(["id"]).transform('mean')

df = df.join(id_means.rename(columns=lambda x: "m_" + x))
df.columns

Index(['id', 'time', 'd', 'y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',
       'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18',
       'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28',
       'x29', 'x30', 'm_d', 'm_x1', 'm_x2', 'm_x3', 'm_x4', 'm_x5', 'm_x6',
       'm_x7', 'm_x8', 'm_x9', 'm_x10', 'm_x11', 'm_x12', 'm_x13', 'm_x14',
       'm_x15', 'm_x16', 'm_x17', 'm_x18', 'm_x19', 'm_x20', 'm_x21', 'm_x22',
       'm_x23', 'm_x24', 'm_x25', 'm_x26', 'm_x27', 'm_x28', 'm_x29', 'm_x30'],
      dtype='object')

In [8]:
# CRE normality assumption
np.random.seed(1)
obj_dml_data_pdml = DoubleMLClusterData(df,
                                        y_col='y',
                                        d_cols='d',
                                        cluster_cols='id',
                                        x_cols=[col for col in df.columns if col.startswith("x")],
                                        use_other_treat_as_covariate=False,
                                        m_d_cols='m_d')

learner = LassoCV()
ml_l = clone(learner)
ml_m = clone(learner)

obj_dml_plr_sim = DoubleMLPLPR(obj_dml_data_pdml, ml_l, ml_m, pdml_approach='cre')
obj_dml_plr_sim.fit()

print(obj_dml_plr_sim.summary)

       coef   std err           t  P>|t|     2.5 %    97.5 %
d  1.384331  0.012053  114.854648    0.0  1.360708  1.407954


In [9]:
# CRE general
np.random.seed(1)
obj_dml_data_pdml = DoubleMLClusterData(df,
                                        y_col='y',
                                        d_cols='d',
                                        cluster_cols='id',
                                        x_cols=[col for col in df.columns if col.startswith("x")],
                                        use_other_treat_as_covariate=False)

learner = LassoCV()
ml_l = clone(learner)
ml_m = clone(learner)

obj_dml_plr_sim = DoubleMLPLPR(obj_dml_data_pdml, ml_l, ml_m, pdml_approach='cre_general')
obj_dml_plr_sim.fit()

print(obj_dml_plr_sim.summary)

       coef   std err           t  P>|t|    2.5 %    97.5 %
d  1.356699  0.011841  114.573199    0.0  1.33349  1.379907


In [10]:
# lasso extended dict

xdat = df.loc[:,df.columns.str.startswith('x')]
xmdat = df.loc[:,df.columns.str.startswith('m_x')]

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2, include_bias=False)

xpoly = poly.fit_transform(xdat)
x_p3 = xdat**3

x_pol_nam = poly.get_feature_names_out()
x_cols_p3 = [f'x{i + 1}^3' for i in np.arange(xdat.shape[1])]

#
xmpoly = poly.fit_transform(xmdat)
xm_p3 = xmdat**3

xm_pol_nam = poly.get_feature_names_out()
xm_cols_p3 = [f'm_x{i + 1}^3' for i in np.arange(xmdat.shape[1])]

X_all = np.column_stack((xpoly, x_p3, xmpoly, xm_p3))

x_df = pd.DataFrame(X_all, columns = list(x_pol_nam) + x_cols_p3 + list(xm_pol_nam) + xm_cols_p3)

df_ext = df[['id', 'time', 'd', 'y', 'm_d']].join(x_df)

df_ext.shape

(2500, 1055)

In [11]:
np.random.seed(1)
obj_dml_data_pdml_ext = DoubleMLClusterData(df_ext,
                                        y_col='y',
                                        d_cols='d',
                                        cluster_cols='id',
                                        x_cols=[col for col in df_ext.columns if "x" in col],
                                        use_other_treat_as_covariate=False)

learner = LassoCV()

ml_l = clone(learner)
ml_m = clone(learner)

obj_dml_plr_sim = DoubleMLPLPR(obj_dml_data_pdml_ext, ml_l, ml_m, pdml_approach='cre_general')

obj_dml_plr_sim.fit(n_jobs_cv=1)

print(obj_dml_plr_sim.summary)

       coef   std err          t          P>|t|     2.5 %   97.5 %
d  0.512186  0.021273  24.076758  4.379828e-128  0.470491  0.55388


In [12]:
np.random.seed(1)
obj_dml_data_pdml_ext = DoubleMLClusterData(df_ext,
                                        y_col='y',
                                        d_cols='d',
                                        cluster_cols='id',
                                        x_cols=[col for col in df_ext.columns if "x" in col],
                                        m_d_cols='m_d',
                                        use_other_treat_as_covariate=False)

learner = LassoCV()

ml_l = clone(learner)
ml_m = clone(learner)

obj_dml_plr_sim = DoubleMLPLPR(obj_dml_data_pdml_ext, ml_l, ml_m, pdml_approach='cre')

obj_dml_plr_sim.fit()

print(obj_dml_plr_sim.summary)

       coef   std err          t         P>|t|     2.5 %    97.5 %
d  0.584661  0.036266  16.121595  1.799055e-58  0.513582  0.655741


In [16]:
obj_dml_data_pdml_ext.m_d

array([[ 3.21774482],
       [ 3.21774482],
       [ 3.21774482],
       ...,
       [-1.35198812],
       [-1.35198812],
       [-1.35198812]], shape=(2500, 1))